In [ ]:
import ast
import json
import os
import sys
import timeit

from collections import Counter


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from pathlib import Path
from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset
import yaml

In [ ]:
basepath = Path('/mnt/alex.li/productivity_dsets/')

In [ ]:
dsetnames = [
    'Jupiter_20231121_HHH2_1800_1830'
]

In [ ]:
memuse = df.memory_usage(deep=True) / 1e9; total = sum(memuse); total

In [ ]:
memuse['weather_summary']

In [ ]:
'\n'.join(list(sorted(list(memuse.items()), key=lambda x:-x[1])))

In [ ]:
df['annotation_pixelwise_0_labeled_objects']

In [ ]:
df[['annotation_pixelwise_0_labeled_objects','annotation_pixelwise_0_labeled_objects_right']].memory_usage()

In [ ]:
dsets = {}
for dsetname in dsetnames:
    print(dsetname)
    path = basepath / dsetname
    if not os.path.exists(path / 'annotations.csv'):
        print(f"Downloading to {path}")
        dset: Dataset = Dataset.retrieve(name=dsetname)
        df = dset.to_dataframe()
        dset.download(path)
        df.to_csv(path / 'annotations.csv')
    else:
        df = pd.read_csv(path / 'annotations.csv')
    dsets[dsetname] = {
        'df': df,
        'dpath': path,
    }


In [ ]:
from dl.dataset.pack_perception.download_ocal_data import download_ocal_data
download_ocal_data(str(path), df)

In [ ]:
from dl.dataset.pack_perception.ml_pack_perception import MLPackPerception
from dl.dataset.pack_perception.pack_perception_parameters import (
    PackPerceptionParameters,
)
dataset_path = basepath / dsetnames[0]
params = PackPerceptionParameters()
args = params.parser.parse_args(args=[
    '--csv-path', str(dataset_path / 'annotations.csv'),
    '--data-dir', str(dataset_path),
    '--cam-calibration-path', str(dataset_path / 'online_calibration_data'),
    '--batch-size', '4',
    '--multiprocess-workers', '32',
    '--pandarallel-workers', '32',
    '--gpu', '0',
    '--models', '512,768=ml_512x768_v3_full_rgb_halo_depth_10062023.ckpt', '512,640=ml_512x640_v3_full_rgb_halo_depth_10062023.ckpt'
    '--model-type' 'full' '--max-disp' '384' \
    '--run-oc''--image-only'
])

# MLPackPerception(vars(args))

In [ ]:
MLPackPerception(**vars(args))

In [ ]:
# Done already by stewart: 
# - Headlands removed
# - 

# Remove images with FPs

# Trim overlong sequences

# Choose sequences to meet criteria?